In [14]:
import json

def preprocess_webnlg(path):
    with open(path,"r") as file:
        data=file.read()
    obj=json.loads(data)
    entries=obj["entries"]
    output_list=list()
    for i in range(10):#len(entries)):
        for lex in list(map(lambda el: el["lex"],entries[i][str((i+1))]["lexicalisations"])):
            print(entries[i][str((i+1))])
            output_list.append({
                "instruction": "The input is a list of triplets. Each triple has a subject, a predicate and an object. Try to produce a sentence out of it.",
                "modified":entries[i][str((i+1))]["modifiedtripleset"],
                "original":"",
                "output": lex
            })
    return output_list

In [ ]:
dataset_base_path="webnlg-dataset-master/"
dataset_version_path="release_v2.1/json/"
train=preprocess_webnlg(dataset_base_path+dataset_version_path+"webnlg_release_v2.1_train.json")
val=preprocess_webnlg(dataset_base_path+dataset_version_path+"webnlg_release_v2.1_dev.json")
test=preprocess_webnlg(dataset_base_path+dataset_version_path+"webnlg_release_v2.1_test.json")
print(len(train))
print(len(val))
print(len(test))

In [8]:
test[1603]

{'instruction': 'The input is a list of triplets. Each triple has a subject, a predicate and an object. Try to produce a sentence out of it.',
 'input': [{'object': 'Daniel_Gould_Fowle',
   'property': 'successor',
   'subject': 'Alfred_Moore_Scales'},
  {'object': 'Princeton_University',
   'property': 'almaMater',
   'subject': 'Daniel_Gould_Fowle'}],
 'output': 'Alfred Moore Scales was succeeded by Daniel Gould Fowle, his alma mater is Princeton University.'}